In [32]:
import matplotlib.pyplot as plt
import networkx as nx
import itertools

In [33]:
class Cell:
    def __init__(self, value:str =None, i:int=None, j:int=None):
        self.value = value
        self.i = i
        self.j = j
        self.x = None
        self.y = None

    def set_pointers(self, cell1, cell2):
        self.x = cell1
        self.y = cell2

    def set_x(self, cell1):
        self.x = cell1

    def set_y(self, cell2):
        self.y = cell2
    
    def __repr__(self):
        pointer1_value = self.x.value if self.x else None
        pointer2_value = self.y.value if self.y else None
        return f"Cell {self.value}"

In [34]:
def get_pos(x, y, size):
    return x+y*size

In [35]:
def check_in_bounds(i, j, x, y):
    return i >= 0 and i < x and j >= 0 and j < y

In [36]:
def set_neighbourhood(rule_l):
    neighbours = [0]*9
    neighbours[0] = rule_l[6]
    neighbours[1] = rule_l[7]
    neighbours[2] = rule_l[8]
    neighbours[3] = rule_l[5]
    neighbours[4] = rule_l[0]
    neighbours[5] = rule_l[1]
    neighbours[6] = rule_l[4]
    neighbours[7] = rule_l[3]
    neighbours[8] = rule_l[2]
    return neighbours

In [37]:
def apply_rule(rule:int, x:int, y:int, image):
    rule_l = list(map(lambda x: int(x), list(format(rule, '09b'))))
    rule_l.reverse()
    neighbours = set_neighbourhood(rule_l)
    for i in range(y):
        for j in range(x):
            # print(image[get_pos(j, i, x)])
            cell = image[get_pos(j, i, x)]
            ct = 0
            for y_ in range(-1,2):
                for x_ in range(-1, 2):
                    if check_in_bounds(j+x_, i+y_, x, y) and neighbours[ct] == 1:
                        neighbour = image[get_pos(j+x_, i+y_, x)]
                        if cell.x is None:
                            cell.set_x(neighbour)
                        elif cell.y is None:
                            cell.set_y(neighbour)
                    ct+=1            


In [38]:
def create_adj_matrix(x:int, y:int, image):
    adj_matrix = [[0 for j in range(len(image))] for i in range(len(image))]
    for i, cell in enumerate(image):
        if cell.x is not None:
            adj_matrix[i][get_pos(cell.x.j-1, cell.x.i-1, x)] = 1
        if cell.y is not None:
            adj_matrix[i][get_pos(cell.y.j-1, cell.y.i-1, x)] = 1
    
    return adj_matrix
        

In [39]:
def find_incoming_nodes(pos:int, image, adj_matrix):
    nodes = []
    for i in range(len(image)):
        if adj_matrix[i][pos] == 1:
            nodes.append(image[i])
    
    return nodes

In [40]:
def find_outgoing_nodes(pos:int, image, adj_matrix):
    return [image[i] for i in range(len(image)) if adj_matrix[pos][i] == 1]


In [41]:
def find_chain(pos:int, x, y, chain:set, frm:Cell, image, adj_matrix):
    chain.add(image[pos])
    
    incoming = find_incoming_nodes(pos, image, adj_matrix)
    incoming = [node for node in incoming if node is not frm]
    for node in incoming:
        outgoing = find_outgoing_nodes(get_pos(node.j-1, node.i-1, x), image, adj_matrix)
        for n in outgoing:
            if n is not image[pos]:
                find_chain(get_pos(n.j-1, n.i-1, x), x, y, chain, node, image, adj_matrix)


In [42]:
def find_max(rule: int, x, y, image, adj_matrix, memory:dict):
    if(rule in memory.keys()):
        return memory[rule]
    
    maxx = 0
    longest_chain = set()
    for i in range(y):
        for j in range(x):
            chain = set()
            find_chain(get_pos(j, i, x), x, y, chain, None, image, adj_matrix)
            if len(chain) > maxx:
                longest_chain = chain
                maxx = len(chain)
    
    print(longest_chain)
    memory[rule] = maxx
    return maxx

In [43]:
def conv_to_c2(rule: int):
    l = list(map(int, reversed(list(format(rule, '09b')))))
    l = [i for i,j in enumerate(l) if j != 0]
    comps = list(map(lambda x:2**x[0]+2**x[1], itertools.combinations(l, 2)))
    
    return comps
    

In [44]:
def find_k(rule: int, x:int, y:int, memory:dict):
    comps = conv_to_c2(rule)
    m = 0
    for r in comps:
        print(f"########## RULE {r} ##########")
        image = [Cell("B"+str(i)+str(j), i, j) for i in range(1,y+1) for j in range(1,x+1)]
        # print(*image, sep="\n")
        apply_rule(r, x, y, image)
        adj_matrix = create_adj_matrix(x, y, image)
        # print(*adj_matrix, sep="\n")
        m = max(m, find_max(r, x, y, image, adj_matrix, memory))

    return m

In [45]:
def calculate(r):
    c = dict()
    rules = sorted(list(map(lambda x: sum(map(lambda y: 2**y, x)), itertools.combinations(range(0,9), r))))
    memory = dict()
    for rule in rules:
        c[rule] = find_k(rule, 3, 7,memory)
    
    return c



In [47]:
c2 = calculate(2)

########## RULE 3 ##########
{Cell B11, Cell B13, Cell B12}
########## RULE 5 ##########
{Cell B33, Cell B22, Cell B11}
########## RULE 6 ##########
{Cell B52, Cell B32, Cell B62, Cell B72, Cell B42, Cell B22, Cell B12}
########## RULE 9 ##########
{Cell B11, Cell B51, Cell B41, Cell B71, Cell B31, Cell B21, Cell B61}
########## RULE 10 ##########
{Cell B13, Cell B22, Cell B31}
########## RULE 12 ##########
{Cell B21, Cell B23, Cell B22}
########## RULE 17 ##########
{Cell B13, Cell B22, Cell B31}
########## RULE 18 ##########
{Cell B13, Cell B21}
########## RULE 20 ##########
{Cell B23, Cell B21}
########## RULE 24 ##########
{Cell B22, Cell B21, Cell B23}
########## RULE 33 ##########
{Cell B13, Cell B12, Cell B11}
########## RULE 34 ##########
{Cell B11, Cell B13}
########## RULE 36 ##########
{Cell B23, Cell B11}
########## RULE 40 ##########
{Cell B22, Cell B11, Cell B33}
########## RULE 48 ##########
{Cell B51, Cell B41, Cell B31, Cell B11, Cell B71, Cell B21, Cell B61}
#########

In [48]:
c3 = calculate(7)

########## RULE 3 ##########
{Cell B13, Cell B12, Cell B11}
########## RULE 5 ##########
{Cell B11, Cell B22, Cell B33}
########## RULE 9 ##########
{Cell B31, Cell B61, Cell B41, Cell B71, Cell B21, Cell B11, Cell B51}
########## RULE 17 ##########
{Cell B31, Cell B13, Cell B22}
########## RULE 33 ##########
{Cell B13, Cell B12, Cell B11}
########## RULE 65 ##########
{Cell B11, Cell B33, Cell B22}
########## RULE 6 ##########
{Cell B32, Cell B42, Cell B22, Cell B12, Cell B62, Cell B72, Cell B52}
########## RULE 10 ##########
{Cell B13, Cell B31, Cell B22}
########## RULE 18 ##########
{Cell B13, Cell B21}
########## RULE 34 ##########
{Cell B13, Cell B11}
########## RULE 66 ##########
{Cell B23, Cell B11}
########## RULE 12 ##########
{Cell B21, Cell B23, Cell B22}
########## RULE 20 ##########
{Cell B21, Cell B23}
########## RULE 36 ##########
{Cell B11, Cell B23}
########## RULE 68 ##########
{Cell B11, Cell B33}
########## RULE 24 ##########
{Cell B23, Cell B21, Cell B22}
########

In [50]:
print(*zip(c3.keys(), c3.values()), sep="\n")

(127, 7)
(191, 7)
(223, 7)
(239, 7)
(247, 7)
(251, 7)
(253, 7)
(254, 7)
(319, 7)
(351, 7)
(367, 7)
(375, 7)
(379, 7)
(381, 7)
(382, 7)
(415, 7)
(431, 7)
(439, 7)
(443, 7)
(445, 7)
(446, 7)
(463, 7)
(471, 7)
(475, 7)
(477, 7)
(478, 7)
(487, 7)
(491, 7)
(493, 7)
(494, 7)
(499, 7)
(501, 7)
(502, 7)
(505, 7)
(506, 7)
(508, 7)


In [53]:
c3 = calculate(3)

########## RULE 3 ##########
{Cell B12, Cell B11, Cell B13}
########## RULE 5 ##########
{Cell B11, Cell B22, Cell B33}
########## RULE 6 ##########
{Cell B12, Cell B72, Cell B32, Cell B42, Cell B22, Cell B62, Cell B52}
########## RULE 3 ##########
########## RULE 9 ##########
{Cell B71, Cell B61, Cell B31, Cell B41, Cell B11, Cell B51, Cell B21}
########## RULE 10 ##########
{Cell B31, Cell B22, Cell B13}
########## RULE 5 ##########
########## RULE 9 ##########
########## RULE 12 ##########
{Cell B22, Cell B23, Cell B21}
########## RULE 6 ##########
########## RULE 10 ##########
########## RULE 12 ##########
########## RULE 3 ##########
########## RULE 17 ##########
{Cell B31, Cell B13, Cell B22}
########## RULE 18 ##########
{Cell B21, Cell B13}
########## RULE 5 ##########
########## RULE 17 ##########
########## RULE 20 ##########
{Cell B21, Cell B23}
########## RULE 6 ##########
########## RULE 18 ##########
########## RULE 20 ##########
########## RULE 9 ##########
########## RU

In [54]:
c3[148]

3